# Scraping tabular data from https://2016halalanresults.abs-cbn.com/ with python.

_author: guintojames8@gmail.com_

_2020-03-05_

_Objective: Extract tabular data from multiple web pages._

**Python Packages**

- pandas - data handling, processing, cleaning.
- bs4 (BeautifulSoup) - extracting information from HTML.
- requests - loading web pages programatically. 

**Scraping procedure followed**

	generate url of each city/town
	
	for each url
		extract all tables
        
**URL generation**

The URL's were automatically generated by observing the url structure of the web pages. 

```
https://<base_url>/<regionid>/<provinceid>/<cityid>/<province_name>.html
```

The url information were extracted from the HTMl code and were structured to follow the structure shown above. 

**Table extraction**

The tables in each web page were extracted using the _HalalanResultScraper_ class. This class was custom built to extract the relevant information needed from each web page. The scraper outputs a list of dictionary with all the relevant information in the _response_ attribute and a list of urls that failed to load in the _failed_urls_ attribute. 


In [256]:
import time, random, json

import requests
from bs4 import BeautifulSoup
import pandas as pd

## URL Generation

In [160]:
with open("Halalan 2016 - Partial and Unofficial Results _ ABS-CBN News.html") as fp:
    cities_soup = BeautifulSoup(fp, "html.parser")

In [186]:
html_links = []
for city in cities_soup.find_all('ul', {'id':'selection'})[0].find_all('a'):
    try:
        html_links.append({
            'regionid':city['regionid'],
            'provinceid':city['provinceid'],
            'cityid':city['cityid'],
            'province_name':city['province-name']
        })
    except:
        continue

In [187]:
html_links_df = pd.DataFrame(html_links)

def clean_province_name(h):
    h = h.lower()
    h = h.replace(',', '')
    h = h.replace(' ','-')
    
    return h

html_links_df['province_name_url'] = html_links_df['province_name'].apply(clean_province_name)
html_links_df['url'] = 'https://2016halalanresults.abs-cbn.com/national/'\
    + html_links_df['regionid'] + '/'\
    + html_links_df['provinceid'] + '/'\
    + html_links_df['cityid'] + '/'\
    + html_links_df['province_name_url'] + '.html'

html_links_df.head()

,regionid,provinceid,cityid,province_name
0,0,0,0,Philippines
1,1,1,0,METRO MANILA
2,1,1,1,"CALOOCAN CITY, METRO MANILA"
3,1,1,2,"CITY OF LAS PIÑAS, METRO MANILA"
4,1,1,3,"CITY OF MAKATI, METRO MANILA"


## Table Extraction

In [248]:
class HalalanResultScraper():
    def __init__(self):
        self.response = []
        self.failed_urls = []
        
        
    def get_page(self, url):
        try:
            page = requests.get(url)
            
            if page.ok == False:
                raise Exception("can't get page")
                
            return BeautifulSoup(page.text, 'lxml')
        
        except Exception as e:
            print(e)
            #raise Exception("can't get page, some error")
        
    def get_results_table(self, soup):
        res = soup.find_all('div', {'class':'result-block'})
        
        if len(res) == 0:
            return False
        
        return res
    
    def get_position(self, soup):
        res = soup.find_all('div', {'class':'title-bar'})[0]\
            .find('h2').text
        
        if len(res) == 0:
            raise Exception("no position")
        
        return res
    
    def get_headers(self, soup):
        res = []
        
        for h in soup.find_all('th'):
            res.append(h.text)

        return res
    
    def get_content(self, soup):
        # get table body
        body = soup.find_all('tbody')
        
        if len(body) == 0:
            raise Exception("can't find table body")
            
        for b in body:
            res = []
            for r in b.find_all('tr'):
                row = []
                for c in  r.find_all('td'):
                    row.append(c.text)
                res.append(row)
                
        return res
        
    def main(self, urls, prevent_ddos=True):
        for i, url in enumerate(urls):
            try:
                # prevent ddos by sleeping 
                if prevent_ddos and i % 10 == 0:
                    time.sleep(random.randint(1,5))
                print(url)
                # load page
                page = self.get_page(url)

                # find all results table
                results_table = self.get_results_table(page)

                # if no results table then continue
                if res == False:
                    self.failed_urls.append(url)
                    continue

                for tbl in results_table:
                    try:
                        # get position of current table
                        position = self.get_position(tbl)

                        # get headers of the table
                        headers = self.get_headers(tbl)

                        content = self.get_content(tbl)

                        self.response.append({
                            'url':url,
                            'position':position,
                            'headers':headers,
                            'content':content
                        })

                    except Exception as e:
                        self.failed_urls.append((url, e))
                        continue
            except Exception as e:
                self.failed_urls.append((url, e))
                continue
            

In [249]:
urls = list(html_links_df['url'])

ab = HalalanResultScraper()
ab.main(urls)

https://2016halalanresults.abs-cbn.com/national/0/0/0/philippines.html
can't get page
https://2016halalanresults.abs-cbn.com/national/1/1/0/metro-manila.html
can't get page
https://2016halalanresults.abs-cbn.com/national/1/1/1/caloocan-city-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/2/city-of-las-piñas-metro-manila.html
can't get page
https://2016halalanresults.abs-cbn.com/national/1/1/3/city-of-makati-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/4/city-of-mandaluyong-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/5/city-of-marikina-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/6/city-of-muntinlupa-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/7/city-of-parañaque-metro-manila.html
can't get page
https://2016halalanresults.abs-cbn.com/national/1/1/8/city-of-pasig-metro-manila.html
https://2016halalanresults.abs-cbn.com/national/1/1/9/city-of-valenzuela-metro-manila.

In [255]:
# export list of failed urls
with open('failed_urls.txt', 'w') as f:
    for txt in ab.failed_urls:
        f.write(txt[0] + '\n')

In [258]:
# export raw data to json
with open("20220305_HalalanResults_raw.json", "w") as outfile:
    json.dump(ab.response, outfile)

## Data Cleaning

In [317]:
def transform_wide_form(row):
    header = row['headers']
    content = row['content']
    
    res = []
    for content_row in content:
        #print(header)
        #print(content_row)
        temp_dict = {}
        for h, c in zip(header, content_row):
            temp_dict[h] = c
            
        temp_dict['position'] = row['position']
        temp_dict['url'] = row['url']
        
        res.append(temp_dict)
    return res

In [318]:
# transform data so that we can load it as a dataframe.
clean_rows = []
for row in ab.response:
    clean_rows += transform_wide_form(row)
    
# convert transformed rows to a dataframe.
halalan_results_df = pd.DataFrame(clean_rows)

# add province variables.
halalan_results_df_wprovince = halalan_results_df.merge(html_links_df, on='url', how='left')

# reorder and rename
output = halalan_results_df_wprovince[['url','regionid', 'provinceid', 'cityid','province_name','position','Candidate','Political Party','Vote Count','Rank']].copy()
output.columns = ['url','region_id', 'province_id', 'city_id','province_name','position','candidate_name','political_party','vote_count','rank']
output.head(5)

,url,region_id,province_id,city_id,province_name,position,candidate_name,political_party,vote_count,rank
0,https://2016halalanresults.abs-cbn.com/nationa...,1,1,1,"CALOOCAN CITY, METRO MANILA",PRESIDENTIAL,"\nDUTERTE, RODY\nView details Â»\n",PARTIDO DEMOKRATIKO PILIPINO LAKAS NG BAYAN,231626,1
1,https://2016halalanresults.abs-cbn.com/nationa...,1,1,1,"CALOOCAN CITY, METRO MANILA",PRESIDENTIAL,"\nPOE, GRACE\nView details Â»\n",INDEPENDENT,115480,2
2,https://2016halalanresults.abs-cbn.com/nationa...,1,1,1,"CALOOCAN CITY, METRO MANILA",PRESIDENTIAL,"\nROXAS, MAR\nView details Â»\n",LIBERAL PARTY,68711,3
3,https://2016halalanresults.abs-cbn.com/nationa...,1,1,1,"CALOOCAN CITY, METRO MANILA",PRESIDENTIAL,"\nBINAY, JOJO\nView details Â»\n",UNITED NATIONALIST ALLIANCE,58799,4
4,https://2016halalanresults.abs-cbn.com/nationa...,1,1,1,"CALOOCAN CITY, METRO MANILA",PRESIDENTIAL,"\nDEFENSOR SANTIAGO, MIRIAM\nView details Â»\n",PEOPLE`S REFORM PARTY,27842,5


In [312]:
# additional cleaning steps
output['candidate_name'] = output['candidate_name'].str.replace('\n','', case=False)
output['candidate_name'] = output['candidate_name'].str.replace('View details Â»','', case=False)
output['candidate_name']

0                    DUTERTE, RODY
1                       POE, GRACE
2                       ROXAS, MAR
3                      BINAY, JOJO
4        DEFENSOR SANTIAGO, MIRIAM
                   ...            
96591           VALEROSO, DIOSDADO
96592                  CAM, SANDRA
96593              BALIGOD, LEVITO
96594         PALPARAN, JOVITO JR.
96595                  KABALU, EID
Name: candidate_name, Length: 96596, dtype: object

In [314]:
# export
output.to_csv('HalalanResults.csv',index=False)